# Trabajar con datos

In [ ]:
! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## Listar los almacenes de datos

In [ ]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

## Crear un datastore

To create a datastore and connect to a (already existing) storage, you'll need to specify:

- La clase para indicar con qué tipo de servicio de almacenamiento desea conectar.El siguiente ejemplo se conecta a un almacenamiento Blob (Azure Blob DataStore).
- name: El nombre del almacén de datos en el espacio de trabajo de Azure Machine Learning.
- description: Descripción opcional para proporcionar más información sobre el almacén de datos.
- account_name: El nombre de la cuenta de almacenamiento de Azure.
- container_name: El nombre del contenedor para almacenar blobs en la cuenta de almacenamiento de Azure.
- credentials: Proporcione el método de autenticación y las credenciales para autenticar.El siguiente ejemplo usa una clave de cuenta.

Reemplazar XXXX-XXXX con la clave de cuenta del storage account.

In [ ]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities import AccountKeyConfiguration

store = AzureBlobDatastore(
    name="blob_training_data",
    description="Blob Storage for training data",
    account_name="YOUR-STORAGE-ACCOUNT-NAME",
    container_name="training-data", 
    credentials=AccountKeyConfiguration(
        account_key="XXXX-XXXX"
    ),
)

ml_client.create_or_update(store)

Listar los almacenes de datos nuevamente para verificar que se haya creado un nuevo almacén de datos llamado `blob_training_data`:

In [ ]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

## Crear data assets

- `URI_FILE` apunta a un archivo específico.
- `URI_FOLDER` apunta a una carpeta específica.
- `MLTABLE` señala un archivo MLTable que especifica cómo leer uno o más archivos dentro de una carpeta.


In [ ]:
# URI_FILE

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './data/diabetes.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Activo de datos que apunta a un archivo local, cargado automáticamente en el almacén de datos predeterminado",
    name="diabetes-local"
)

ml_client.data.create_or_update(my_data)

In [38]:
# URI_FOLDER

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

datastore_path = 'azureml://datastores/blob_training_data/paths/data-asset-path/'

my_data = Data(
    path=datastore_path,
    type=AssetTypes.URI_FOLDER,
    description="Activo de datos que apunta a la carpeta de datos de datos en el almacén de datos",
    name="diabetes-datastore-path"
)

ml_client.data.create_or_update(my_data)

Data({'path': 'azureml://subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4/datastores/blob_training_data/paths/data-asset-path/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-datastore-path', 'description': 'Activo de datos que apunta a la carpeta de datos de datos en el almacén de datos', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/data/diabetes-datastore-path/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/formacionvm/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0

In [ ]:
# MLTABLE
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_path = 'data/'

my_data = Data(
    path=local_path,
    type=AssetTypes.MLTABLE,
    description="Mltable apuntando a diabetes.csv en la carpeta de datos",
    name="diabetes-table"
)

ml_client.data.create_or_update(my_data)

Para verificar que se hayan creado los nuevos activos de datos, se puede enumerar todos los activos de datos en el espacio de trabajo nuevamente:

In [ ]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

## Leer datos en notebook

- Instalar `mltable`, luego convertir el activo de datos en un dataframe y visualizarlo.

In [ ]:
import mltable

registered_data_asset = ml_client.data.get(name='diabetes-table', version=1)
tbl = mltable.load(f"azureml:/{registered_data_asset.id}")
df = tbl.to_pandas_dataframe()
df.head(5)

## Usar datos en un job

In [ ]:
import os

# Crea una carpeta para los archivos de script
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

In [36]:
%%writefile $script_folder/move-data.py
# import libraries
import argparse
import pandas as pd
import numpy as np
from pathlib import Path

def main(args):
    # read data
    print('reading from:', args.input_data)
    df = get_data(args.input_data)

    print('moving to:', args.output_datastore)


    output_df = df.to_csv((Path(args.output_datastore) / "diabetes.csv"), index = False)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Analyzing {} rows of data'.format(row_count))
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_datastore", dest='output_datastore',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Overwriting src/move-data.py


Enviar un job que ejecute el script **move-data.py**

In [ ]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import command

# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-local:1")
}

my_job_outputs = {
    "datastore_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/blob_training_data/paths/data-asset-path/")
}


# configure job
job = command(
    code="./src",
    command="python move-data.py --input_data ${{inputs.local_data}} --output_datastore ${{outputs.datastore_data}}",
    inputs=my_job_inputs,
    outputs=my_job_outputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="<vm o cluster>",
    display_name="move-diabetes-data",
    experiment_name="move-diabetes-data"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Monitor your job at https://ml.azure.com/runs/frosty_sponge_dwxmksbyc9?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156
